In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
# ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-130881
Azure region: southcentralus
Subscription id: 3929502f-a035-4029-a2f5-865565e57a0a
Resource group: aml-quickstarts-130881


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found an existing compute target.')
except ComputeTargetException:
    print('No existing compute target found. Creating a new compute target.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

print(compute_target.get_status().serialize())

No existing compute target found. Creating a new compute target.
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-12-18T11:17:35.355000+00:00', 'errors': None, 'creationTime': '2020-12-18T11:17:31.105043+00:00', 'modifiedTime': '2020-12-18T11:17:47.622700+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [21]:
from azureml.core import ScriptRunConfig, Environment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, normal, uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
     {
        '--C': choice(1,2,4), 
        '--max_iter': choice(10,50,100)
     }
)

# Specify a Policy
policy = BanditPolicy(
           slack_factor = 0.1,
           evaluation_interval = 2
)

# if "training" not in os.listdir():
#    os.mkdir("./training")

env = Environment(name="myenv")

# Create an SKLearn estimator for use with train.py
# The SKLearn feature has been deprecated according to Dec 7 release notes
# Therefore ScriptRunConfig is used here instead
# est = SKLearn(source_directory='./training', compute_target='cluster_name', entry_script='train.py')

config = ScriptRunConfig(source_directory='./',
                         script='train.py',
                         compute_target=compute_target,
                         environment=env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=40,
                             max_concurrent_runs=4)

In [22]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)

from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_2a6943a3-faf5-4afc-ba06-966266247d5e
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_2a6943a3-faf5-4afc-ba06-966266247d5e?wsid=/subscriptions/3929502f-a035-4029-a2f5-865565e57a0a/resourcegroups/aml-quickstarts-130881/workspaces/quick-starts-ws-130881

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-18T12:03:23.927097][API][INFO]Experiment created<END>\n""<START>[2020-12-18T12:03:24.391454][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2020-12-18T12:03:24.7125788Z][SCHEDULER][WARNING]An error occurred while trying to prepare the execution environment. The experiment will be marked as failed.<END>"<START>[2020-12-18T12:03:26.553684][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2020-12-18T12:03:29.356708][CONTROLLER][ERROR]Hyperdrive is unable to further process the experiment due to some internal error. Experiment has been marked as failed. 

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Microsoft.HyperDrive.Scheduler.AML.Client.AMLClientException - Message: \"{\"Error\":{\"Code\":\"UserError\",\"Severity\":null,\"Message\":\"Unknown compute target 'cluster_name'.\",\"MessageFormat\":\"Unknown compute target '{TargetName}'.\",\"MessageParameters\":{\"TargetName\":\"cluster_name\"},\"ReferenceCode\":null,\"DetailsUri\":null,\"Target\":null,\"Details\":[],\"InnerError\":{\"Code\":\"BadArgument\",\"InnerError\":{\"Code\":\"UnknownTargetType\",\"InnerError\":null}},\"DebugInfo\":null},\"Correlation\":{\"operation\":\"b7c23b9ca91e7341802a20422a55f5d9\",\"request\":\"becf8c541935bc44\"},\"Environment\":\"southcentralus\",\"Location\":\"southcentralus\",\"Time\":\"2020-12-18T12:03:24.6759666+00:00\",\"ComponentName\":\"execution\"}\", StatusCode: BadRequest, StackTrace: , InnerException: null",
        "messageParameters": {},
        

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best run:', best_run)
print('Metrics:', best_run_metrics)

model = best_run.register_model(model_name='hyperdrive-bank-telemarketing', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_url)


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Split data into train and test sets.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'accuracy',
    training_data = ds,
    label_column_name = 'y',
    n_cross_validations = 5)

In [2]:
# Submit your automl run

experiment = Experiment(ws, "bank_telemarketing_automl")
print("Experiment created")
# submit(config, tags=None, **kwargs)
run = experiment.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model
import onnxruntime, onnxmltools

joblib.dump(fitted_model, "fitted_automl_model.joblib")

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

print(best_run.get_file_names())
print('Best Rund ID: ', best_run.id)
print('\n Accuracy: ', best_run_metric['accuracy'])

model = best_run.register_model(model_name='automl-bank-telemarketing', model_path='outputs/model.joblib')

# session = onnxruntime.InferenceSession("path to model")

In [ ]:
# Delete the cluster instance
azureml.core.compute.AmlCompute.delete()